In [2]:
import numpy as np
import torch
from tqdm import tqdm
import time

from env import MAXFSEnv,MAXSATEnv
from dka_agent import DBAAgent
from gcnn_agent import BipartiteAgent

In [4]:
device = "cuda:1"
agent = DBAAgent(
        input_dim=3,
        hidden_dim=128,
        ff_dim=512,
        k_dim=64,
        v_dim=64,
        n_head=3,
        n_layers=4,
        device=device,
    )
agent.load_state_dict(torch.load("/opt/home/mehdi.zouitine/rl_max_fs/models/best_model_029xvjjv_10_200_9502720_-10.276666666666667.pth",weights_only=True))

<All keys matched successfully>

In [5]:
agent.eval()
agent.to(device)

DBAAgent(
  (projector): Linear(in_features=3, out_features=128, bias=True)
  (layers): ModuleList(
    (0-3): 4 x DBALayer(
      (attention_var): Attention(
        (proj_q): Linear(in_features=128, out_features=192, bias=False)
        (proj_k): Linear(in_features=128, out_features=192, bias=False)
        (proj_v): Linear(in_features=128, out_features=192, bias=False)
        (proj_output): Linear(in_features=192, out_features=128, bias=False)
      )
      (attention_cons): Attention(
        (proj_q): Linear(in_features=128, out_features=192, bias=False)
        (proj_k): Linear(in_features=128, out_features=192, bias=False)
        (proj_v): Linear(in_features=128, out_features=192, bias=False)
        (proj_output): Linear(in_features=192, out_features=128, bias=False)
      )
      (ff_var): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=128, bias=True)
      )
      (ff_cons):

In [25]:
gcnn_agent = BipartiteAgent(
            cons_nfeats=3,
            edge_nfeats=1,
            var_nfeats=1,
            emb_size=128,
            device=device,
            n_layers=4,
        )
gcnn_agent.load_state_dict(torch.load("/opt/home/mehdi.zouitine/rl_max_fs/models/best_model_g8rkkjul_30_150_7864320_-36.4.pth",weights_only=True))
gcnn_agent.eval()
gcnn_agent.to(device);

In [7]:
with torch.no_grad():
    env = MAXSATEnv(300, 60)
    all_r = []
    start = time.time()
    for k in tqdm(range(300)):
        obs, info = env.reset(seed=k)
        done = False
        r = 0
        while not done:

            logits = agent(torch.Tensor(obs["constraint_features"]).unsqueeze(0).to(device),torch.Tensor(obs["edge_features"]).unsqueeze(0).to(device),torch.Tensor(obs["mask"]).unsqueeze(0).to(device),)
            action = torch.argmax(logits, dim=1)
            obs, reward, done, _, _ = env.step(action)
            r += reward
        all_r.append(r)
    print("Time: ", (time.time()-start)/300)

  0%|          | 0/300 [00:00<?, ?it/s]

 49%|████▊     | 146/300 [00:19<00:22,  6.88it/s]

In [32]:
mean_r = np.mean(all_r)
std_r = np.std(all_r)

In [ ]:
with torch.no_grad():
    env = MAXSATEnv(1000, 80)
    all_r = []
    start = time.time()
    for k in tqdm(range(300)):
        obs, info = env.reset(seed=k)
        done = False
        r = 0
        while not done:

            logits = agent(torch.Tensor(obs["constraint_features"]).unsqueeze(0).to(device),torch.Tensor(obs["edge_features"]).unsqueeze(0).to(device),torch.Tensor(obs["mask"]).unsqueeze(0).to(device),)
            action = torch.argmax(logits, dim=1)
            obs, reward, done, _, _ = env.step(action)
            r += reward
        all_r.append(r)
    print("Time: ", (time.time()-start)/300)